# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-20 04:55:55] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-20 04:55:55] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-20 04:55:55] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-20 04:55:58] INFO server_args.py:1669: Attention backend not specified. Use fa3 backend by default.


[2026-01-20 04:55:58] INFO server_args.py:2568: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=40 avail_mem=10.89 GB):  50%|█████     | 10/20 [00:00<00:00, 18.55it/s]

Capturing batches (bs=16 avail_mem=10.87 GB):  65%|██████▌   | 13/20 [00:00<00:00, 17.07it/s]

Capturing batches (bs=8 avail_mem=10.86 GB):  80%|████████  | 16/20 [00:01<00:00, 16.12it/s] 

Capturing batches (bs=1 avail_mem=10.85 GB):  90%|█████████ | 18/20 [00:01<00:00, 10.76it/s]

Capturing batches (bs=1 avail_mem=10.85 GB): 100%|██████████| 20/20 [00:01<00:00, 12.28it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Anon.
I am a German citizen and I have been living in China for over five years now. I am a writer, I have been writing stories for a long time and I have many friends who are writers, I am usually very friendly, always smiling.
What can you do for me?
My goal is to achieve a balance between literature and the creative arts, to explore what I love, to write about things that I care about. I also write articles and short stories, often in Chinese. I have also translated foreign works and can talk about it.
What is your personal background?
I grew up in a small town in the
Prompt: The president of the United States is
Generated text:  very busy. He is always ________ to his work.
A. trying
B. to try
C. trying to
D. trying
Answer:

D

Which of the following statements about an inverter is correct?
A. An inverter is a device that converts AC power to DC power.
B. An inverter can convert AC power to DC power, but it cannot convert DC power to AC po

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Occupation]. I'm a [Skill or Hobby] enthusiast who loves to [Describe a hobby or interest]. I'm always looking for new experiences and adventures, and I'm always eager to learn and grow. I'm a [Favorite Book or Movie] fan and I love to [Describe a favorite book or movie]. I'm also a [Favorite Music] lover and I love to [Describe a favorite song or album]. I'm a [Favorite Sport] enthusiast and I love to [Describe a favorite sport]. I'm a [Favorite Animal] lover and I love to

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. It is the largest city in France and the third-largest city in the European Union. Paris is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and Louvre Museum. It is also a major center for art, culture, and politics in France. Paris is a popular tourist destination and a major economic hub. The city is home to many famous French artists, writers, and musicians. It is also a major center for business and finance in France. Paris is a cultural and historical center that has played a significant role in shaping French identity and politics. It is a major transportation hub and a

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends:

1. Increased automation and robotics: As AI technology continues to advance, we are likely to see an increase in automation and robotics in various industries. This could lead to the creation of more efficient and cost-effective solutions, but it could also lead to job displacement for humans.

2. AI ethics and privacy: As AI technology becomes more advanced, there will be a need to address ethical and privacy concerns. This could lead to the development of new AI ethics standards and regulations, as well



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a computer scientist, specializing in artificial intelligence. I'm excited to work with [Company Name] and be a part of the team that makes [Product Name]. Thank you. [Name] [Phone number] [Email address] [LinkedIn profile if available] [Company name] [Product name] [Company mission or vision statement if available] [Company logo if available] I'm an AI assistant who specializes in artificial intelligence. I'm excited to help you with any questions you have. [Name] [Phone number] [Email address] [LinkedIn profile if available] [Company name] [Product name

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, an iconic French city known for its rich history, vibrant culture, and annual cultural events such as the Eiffel Tower and the Louvre Museum.

The capital of France is Paris, an iconic Fre

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 could

 also

 be

 used

 to

In [6]:
llm.shutdown()